In [1]:
!pip install librosa

In [2]:
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

import os
import keras
import h5py
import librosa
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from collections import OrderedDict

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D

Using TensorFlow backend.


# Read the data

> Helper functions to assist the process to read songs, split then and return an array of spectrograms/melspectrograms

In [3]:
"""
@description: Method to split a song into multiple songs using overlapping windows
"""
def splitsongs(X, y, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []
    temp_y = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)
        temp_y.append(y)

    return np.array(temp_X), np.array(temp_y)

In [4]:
"""
@description: Method to convert a list of songs to a np array of melspectrograms
"""
def to_melspectrogram(songs, n_fft = 1024, hop_length = 512):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft = n_fft,
        hop_length = hop_length)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    return np.array(list(tsongs))

In [5]:
def read_data(src_dir, genres, song_samples, spec_format, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_specs = []
    arr_genres = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        print(folder)
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                print(file_name)
                signal, sr = librosa.load(file_name)
                signal = signal[:song_samples]
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                
                # Convert to dataset of spectograms/melspectograms
                signals, y = splitsongs(signal, genres[x])
                
                # Convert to "spec" representation
                specs = spec_format(signals)
                
                # Save files
                arr_genres.extend(y)
                arr_specs.extend(specs)
                
                
    return np.array(arr_specs), np.array(arr_genres)

In [6]:
# # Parameters
# gtzan_dir = './GTZAN/genres/training_set/'
# song_samples = 660000
# genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4,
#           'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [7]:
# Parameters
gtzan_dir = '../GTZAN/genres/training_set/'
song_samples = 660000
genres = {'blues': 0, 'classical': 1, 'hiphop': 2, 'jazz': 3, 'disco':4, 'rock':5, 'country':6}

In [ ]:
# # Read the data
# X, y = read_data(gtzan_dir, genres, song_samples, to_melspectrogram, debug=False)
# np.save('x_gtzan_npy.npy', X)
# np.save('y_gtzan_npy.npy', y)

In [ ]:
X = np.load('x_gtzan_npy.npy')
y = np.load('y_gtzan_npy.npy')

In [ ]:
# One hot encoding of the labels
y = to_categorical(y)

# Dataset Split

In [ ]:
X_stack = np.squeeze(np.stack((X,) * 3, -1))
X_stack.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stack, y, test_size=0.1, random_state=42, stratify = y)

In [ ]:
# Histogram for train and test 
values, count = np.unique(np.argmax(y_train, axis=1), return_counts=True)
plt.bar(values, count)

values, count = np.unique(np.argmax(y_test, axis=1), return_counts=True)
plt.bar(values, count)
plt.show()

# Training

In [ ]:
# Model Definition

model_name = 'CNN'
input_shape = X_train[0].shape

num_genres = 7

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',input_shape=input_shape))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(128, (3, 3)))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(256, (3, 3)))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(512, (3, 3)))
#model.add(LeakyReLU(alpha=0.1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(num_genres))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model_path = './CNN.h5'

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

batch_size = 128
epochs = 50

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

model_history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks = [checkpoint, earlystop])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("val_loss = {:.3f} and val_acc = {:.3f}".format(score[0], score[1]))

In [ ]:
plt.figure(figsize=(15,7))

plt.subplot(1,2,1)
plt.plot(model_history.history['acc'], label='train')
plt.plot(model_history.history['val_acc'], label='validation')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(model_history.history['loss'], label='train')
plt.plot(model_history.history['val_loss'], label='validation')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
preds = np.argmax(model.predict(X_test), axis = 1)
y_orig = np.argmax(y_test, axis = 1)
cm = confusion_matrix(preds, y_orig)

In [ ]:
keys = OrderedDict(sorted(genres.items(), key=lambda t: t[1])).keys()

plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, keys, normalize=True)

# Test data process

In [8]:
#load model
model = load_model('./CNN.h5')

In [9]:
# Parameters
test_dir = '../GTZAN/genres/testing_set'
song_samples = 660000

In [10]:
def create_filename_list(src_dir):
    filename_list = []
    for root, subdirs, files in os.walk(src_dir):
        for file in files:
            filename_list.append(file)
    return filename_list

In [11]:
def read_test_data(src_dir, song_samples, spec_format):
    arr_specs = []
    music_splits_len = []
    
    for root, subdirs, files in os.walk(src_dir):
        for file in files:
            file_name = src_dir + "/" + file
            signal, sr = librosa.load(file_name)
            
            # target_samples = signal.shape[0] - (signal.shape[0] % sr)
            signal = signal[:song_samples]
                        
            # Convert to dataset of spectograms/melspectograms
            signals = splitsongs_for_test(signal, music_splits_len)
                
            # Convert to "spec" representation
            specs = spec_format(signals)
                
            # Save files
            arr_specs.extend(specs)
    return np.array(arr_specs), music_splits_len

In [12]:
"""
@description: Method to split a song into multiple songs using overlapping windows
"""
def splitsongs_for_test(X, music_splits_len, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)
    music_splits_len.append(len(temp_X))
    return np.array(temp_X)

In [13]:
test_x, music_splits_len = read_test_data(test_dir, song_samples, to_melspectrogram)

In [14]:
# Read the data
np.save('test_x_v3_npy.npy', test_x)

In [15]:
X_test = np.load('test_x_v3_npy.npy')

In [16]:
X_test_stack = np.squeeze(np.stack((X_test,) * 3, -1))

In [17]:
predict_result = model.predict(X_test_stack)

In [18]:
def group_predict_result(predict_result, music_splits_len):
    grouping_predict_result = []
    delete_result = predict_result
    for length in music_splits_len:
        temp_grouping = []
        for i in range(length):
            temp_grouping.append(delete_result[0])
            delete_result = np.delete(delete_result, [0], axis=0)
        grouping_predict_result.append(temp_grouping)
    return grouping_predict_result

In [19]:
def most_frequent(List): 
    counter = 0
    num = List[0]
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i
    return num

In [20]:
def get_predict_result(grouping_predict):
    final_result = []
    for group in grouping_predict:
        group_indexs = []
        for item in group:
            max = np.argmax(item, axis=0)
            group_indexs.append(max.tolist())
        max_duplicated_value = most_frequent(group_indexs)
        final_result.append(max_duplicated_value)
    return final_result

In [21]:
grouping_predict = group_predict_result(predict_result, music_splits_len)

In [22]:
final_predict = get_predict_result(grouping_predict)

In [23]:
filename_list = create_filename_list(test_dir)

In [24]:
predict = pd.DataFrame({"id":filename_list, "class":final_predict})

In [25]:
predict.to_csv("v3_submission.csv", index = False)

In [26]:
predict

,id,class
0,hiphop.00092.au,2
1,rock.00095.au,6
2,country.00097.au,6
3,disco.00094.au,4
4,country.00096.au,6
5,blues.00095.au,0
6,classical.00091.au,1
7,hiphop.00099.au,2
8,hiphop.00094.au,2
9,disco.00092.au,4


In [27]:
genres = {'blues': 0, 'classical': 1, 'hiphop': 2, 'jazz': 3, 'disco':4, 'rock':5, 'country':6}

In [28]:
# genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4,
#           'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [29]:
def getLables():
    labels = []
    for filename in filename_list:
        str_list = filename.split('.')
        labels.append(str_list[0])
    return labels

In [30]:
labels = getLables()

In [31]:
solution = pd.DataFrame({"id":filename_list, "class_truth":labels})

In [32]:
mapping = genres

In [33]:
solution['class_truth'] = solution['class_truth'].map(mapping)

In [34]:
total_count = len(solution['class_truth'])

In [35]:
def caculate_accuracy(total_count):
    correct = 0
    for i in range(total_count):
        print('==============================================================')
        print('solution index ' + str(i) + ' = ' + str(solution['class_truth'][i]))
        print('predict index ' + str(i) + ' = ' + str(predict['class'][i]))
        if(solution['class_truth'][i] == predict['class'][i]):
            print('predict correct')
            correct += 1
    return correct / total_count

In [36]:
caculate_accuracy(total_count)

solution index 0 = 2
predict index 0 = 2
predict correct
solution index 1 = 5
predict index 1 = 6
solution index 2 = 6
predict index 2 = 6
predict correct
solution index 3 = 4
predict index 3 = 4
predict correct
solution index 4 = 6
predict index 4 = 6
predict correct
solution index 5 = 0
predict index 5 = 0
predict correct
solution index 6 = 1
predict index 6 = 1
predict correct
solution index 7 = 2
predict index 7 = 2
predict correct
solution index 8 = 2
predict index 8 = 2
predict correct
solution index 9 = 4
predict index 9 = 4
predict correct
solution index 10 = 4
predict index 10 = 4
predict correct
solution index 11 = 5
predict index 11 = 5
predict correct
solution index 12 = 3
predict index 12 = 3
predict correct
solution index 13 = 0
predict index 13 = 0
predict correct
solution index 14 = 3
predict index 14 = 3
predict correct
solution index 15 = 5
predict index 15 = 5
predict correct
solution index 16 = 2
predict index 16 = 2
predict correct
solution index 17 = 0
predict ind

0.8714285714285714

In [37]:
compare = pd.concat([solution, predict.iloc[:, 1]], axis=1)

In [38]:
compare

,id,class_truth,class
0,hiphop.00092.au,2,2
1,rock.00095.au,5,6
2,country.00097.au,6,6
3,disco.00094.au,4,4
4,country.00096.au,6,6
5,blues.00095.au,0,0
6,classical.00091.au,1,1
7,hiphop.00099.au,2,2
8,hiphop.00094.au,2,2
9,disco.00092.au,4,4


In [39]:
comp = pd.DataFrame(compare)
comp.to_csv('./CNN_comp.csv', index=False)